In [1]:
import numpy as np

# Feed forward

## Data Matrices

For the fully connected feed forward neural network, we have multiple important matrices to keep track of. Let's first define the data matrices for the undefined supervised learning problem that our network will be training on. 

$$X = \begin{bmatrix}
    x_{1,1} & \cdots & x_{1,m} \\
    \vdots & \ddots & \vdots \\
    x_{n_x,1} & \cdots & x_{n_x,m}
\end{bmatrix}$$

Here, $n_x$ is the number of features in the input dataset. $m$ is the number of training examples we will feed into the network. So $X$ can be thought of a collection of column vectors where each one represents a datapoint.

$\underline{Y}$ is defined similarly, where this is the matrix that contains the labels for the training data. 

$$\underline{Y} = \begin{bmatrix}
    y_1 & \cdots & y_i \cdots & y_m
\end{bmatrix}$$

$y_i$ is intentionally left ambiguous here, as it will vary depending on the problem. If it is regression or binary classification, it will be a number, if it is multiclass classification, it will be a vector.

In [2]:
# For now, invent meaningless data that is already normalised for development.
X = np.random.randn(10, 1000) # 100 features, 1000 examples
Y = np.random.randint(2, size=(1,1000)) # the training labels for binary classification for 1000 examples
nx = X.shape[0]
m = X.shape[1]
ny = Y.shape[0]
print("The number of features are ", nx)
print("The number of training examples are ", m)

The number of features are  10
The number of training examples are  1000


## Layer Matrices

### Layer 1

The transformations of the data at each layer are defined by a weight and a bias matrix. These define the sum of linear regressions that are happening at each layer in each neuron in the network. 

For the first hidden layer, the feature matrix $X$ undergoes the linear transformation 

$$Z^{[1]} = W^{[1]}X + B^{[1]}$$

where $W^{[1]}X$ is the weight matrix of the first layer of dimension $(n_{h}^{[1]}, n_x)$ where $n_{h}^{[1]}$ is the number of neurons in the first hidden layer. This ensures that the product $W^{[1]}X$ has the dimension $(n_{h}^{[1]}, m)$, effectively propagating the training samples through the network. 

$B^{[1]}$ is actually a column vector of dimension $(n_{h}^{[1]}, 1)$ that is broadcast to have the dimension $(n_{h}^{[1]}, m)$ to make the calculations easier. 

A non-linear activation function $g^{[1]}(Z^{[1]})$ is applied to the linear transformation $Z^{[1]}$ to give the output of the first layer, 

$$A^{[1]} = g^{[1]}(Z^{[1]}) = g^{[1]}(W^{[1]}X + B^{[1]})$$

$g^{[1]}$ is usually the ReLU function for hidden layers. 

### All Layers

In general, the output of any layer $l$ within the neural network is given by 

$$A^{[l]} = g^{[l]}(Z^{[l]}) = g^{[l]}(W^{[l]}A^{[l-1]} + B^{[1]})$$

where $A^{[l]}$ is of shape $(n_h^{[l]}, m)$ and $W^{[l]}$ is of shape $(n_h^{[l]}, n_h^{[l-1]})$. 

$ l \in [0, L]$ where $l=0$ corresponds to the input layer and $l=L$ is the ouput layer where $A^{[L]} = \hat{Y}$ which are the predictions of the network for the labels $Y$.

When initialising, $B$ can be $0$, but the weight matrices $W$ must not be set to $0$ to ensure that there is symmetry breaking and the network can learn. 

In [3]:
# Let us imagine a 4 hidden layer neural network
layer_dims = [nx, 5, 4, 3, ny]
L = len(layer_dims) - 1

# Initialise the weights and biases, storing in a parameters dictionary
parameters = {}
for l in range (1, L+1):
    parameters[f"W{l}"] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
    parameters[f"B{l}"] = np.zeros((layer_dims[l], m))

In [4]:
# Build the needed activation functions
def relu(Z):
    return np.maximum(0, Z)

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

In [5]:
# Cache to store the linear sum and activation matrices 
cache = {}
# Now calculate the activations - forward propagation through the network
cache['A0'] = X

# All hidden layers use ReLU
for l in range(1, L):
    cache[f"Z{l}"] = np.dot(parameters[f"W{l}"], cache[f"A{l-1}"]) + parameters[f"B{l}"]
    cache[f"A{l}"] = relu(cache[f"Z{l}"])

# Do final layer with sigmoid 
cache[f"Z{L}"] = np.dot(parameters[f"W{L}"], cache[f"A{L-1}"]) + parameters[f"B{L}"]
cache[f"A{L}"] = sigmoid(cache[f"Z{L}"])

# Cost functions

We have now completed our first pass of propagating the training data through the neural net. We need some way of quantifying how good or bad the neural net was at predicting the labels $Y$ across the entire training set $m$. This is the job of the cost function. 

For binary classification, the cost function is given by 
$$J = -\frac{1}{m}\sum_{\text{axis}=1}\left[\vec{Y}\odot\log(\vec{A}^{[L]})+(1-\vec{Y})\odot\log(1-\vec{A}^{[L]})\right]$$

For multiclass classification, the cost function is given by 
$$J = -\frac{1}{m}\sum_{\text{axis}=0}\sum_{\text{axis}=1}\left[Y\odot\log(A^{[L]})\right]$$

For regression, the cost function is given by 
$$J = \frac{1}{2m}\sum_{\text{axis}=1}\left[\vec{A}^{[L+1]}-\vec{Y}\right]^2

In [6]:
def compute_cost(A, Y, type):
    m = Y.shape[1]

    if type == 'binary classification':
        cost = (-1/m) * np.sum((Y * np.log(A) + (1-Y) * np.log(1-A)), axis=1)
    elif type == 'multilabel classification':
        cost = (-1/m) * np.sum(np.sum((Y * np.log(A)), axis=1), axis=0)
    elif type == 'regression':
        cost = (1/2*m) * np.sum((A - Y)**2, axis=1)
    else:
        return ValueError("Please enter a valid problem type of binary classification, multilabel classification, or regression")

    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    cost = float(cost)
    return cost

In [7]:
J = compute_cost(cache[f"A{L}"], Y, 'binary classification')
print("Cost: " + str(J))

Cost: 0.693147182701006


# Backpropagation

The idea of backpropagation is that we use informtion about the derivative of the cost function with respect to each of the weights and biases parameters to update each of these parameters such that they reduce the cost function and the neural network 'learns'. This called **gradient descent** and is done via

$$
\begin{array}{cc}
    w_{kj} \xleftarrow{} \left(w_{kj} - \alpha \frac{\partial J}{\partial w_{jk}}\right) & \quad \quad b_{j} \xleftarrow{} \left(b_{j} - \alpha \frac{\partial J}{\partial b_{j}}\right)
\end{array}
$$

From now on, we will use $\partial \theta$ to represent $\frac{\partial J}{\partial \theta}$ for arbitrary parameter $\theta$.

To perform gradient descent, we need access to these partial derivatives. We lay out four important equations that will allow us to update every parameter in the network

### The Backpropagation Partial Derivatives

$$\partial Z^{[l]} = \partial A^{[l]} \odot \dot{g}^{[l]}(Z^{[l]})$$

$$
\begin{array}{cc}
    \partial W^{[l]} = \partial Z^{[l]}A^{[l-1]T} & \quad \text{and} \quad \partial b^{[l]} = \sum_{\text{axis}=1}\partial{Z^{[l]}}
\end{array}
$$

$$ \partial A^{[l-1]} = W^{[l]\text{T}} \partial Z^{[l]} $$

### Using the Cost Functions

We see that at any given point we need $\partial Z^{[l]}$. Hence, this means we need $\partial A^{[l]}$ and $\dot{g}^{[l]}(Z^{[l]})$. For $l=L$ we can work out $\partial A^{[l]}$ for each cost function

#### Binary Classification & Sigmoid
$$
\begin{array}{cc}
    \partial A^{[L]} = -\frac{1}{m}\left[\frac{Y}{A^{[L]}}-\frac{1-Y}{1-A^{[L]}}\right] & \quad \text{and} \quad \dot{g}^{[L]}(Z^{[L]}) = \frac{e^{-Z}}{\left[1+e^{-Z}\right]^2}
\end{array}
$$

#### Multilabel Classification & Softmax
$$
\begin{array}{cc}
    \partial A^{[L]} = -\frac{1}{m}\frac{Y}{A^{[L]}} & \quad \text{and} \quad \dot{g}^{[L]}(Z^{[L]}) = \frac{e^{Z^{[L]}}}{\sum_{\text{axis}=0}e^{Z^{[L]}}} - \left(\frac{e^{Z^{[L]}}}{\sum_{\text{axis}=0}1+e^{Z^{[L]}}}\right)^2
\end{array}
$$

#### Regression & ReLU
$$
\begin{array}{cc}
    \partial A^{[L]} = -\frac{1}{m}\left(Y-A^{[L]}\right) & \quad \text{and} \quad \dot{g}^{[l]}(Z^{[l]}) = \begin{cases}
    1 & \text{if } Z^{[l]} > 0 \\
    0 & \text{if } Z^{[l]} < 0
\end{cases}
\end{array}
$$

In [8]:
def dA_final(Y, AL, type):
    """Returns a matrix where each element is the derivative of the cost function wrt each element of the final layer activation matrix."""
    m = Y.shape[1]

    if type == 'binary classification':
        dA = -(1/m) * ( ( Y/AL ) - ( 1-Y / 1-AL ) )
    elif type == 'multilabel classification':
        dA = -(1/m) * ( Y/AL )
    elif type == 'regression':
        dA = -(1/m) * ( Y - AL )
    else:
        raise ValueError("Please enter a valid problem type of binary classification, multilabel classification, or regression")
    
    return dA


def activation_prime(Z, type):
    """Return the value of the derivative of the activation function specified at Z."""
    if type == 'sigmoid':
        gprime = np.exp(-Z)/(1+np.exp(-Z))**2
    elif type == 'softmax':
        softmax = np.exp(Z)/np.sum(np.exp(Z), axis=0, keepdims=True)
        gprime = softmax - softmax**2
    elif type == 'relu':
        # Boolean array
        gprime = (Z > 0).astype(int)
    else:
        raise ValueError("Please enter an activation function type of sigmoid, softmax or relu.")
    
    return gprime


def dA_hidden(W_plus_one, dZ_plus_one):
    """Returns a matrix where each element is the derivative of the cost function wrt each element of a hidden layer activation matrix using the recursion relationship.

    Args:
        W_plus_one (np.array): The weight matrix of the layer one step ahead of this layer.
        dZ_plus_one (np.array): The matrix of derivatives of the cost function wrt linear regressions of the layer one step ahead of this one.

    Returns:
        np.array: the value of dA for this current hidden using the recursion relationship.
    """
    return np.dot(W_plus_one.T, dZ_plus_one)

In [9]:
learning_rate = 0.01

# Get dZL
dZL = dA_final(Y=Y, AL=cache[f'Z{L}'], type='binary classification') * activation_prime(cache[f"Z{L}"], 'sigmoid')
# Get dWL
dWL = np.dot(dZL, cache[f"A{L-1}"].T)
# Get dBL
dBL = np.sum(dZL, axis=1, keepdims=True)

/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_7311/1066190070.py:6: RuntimeWarning: divide by zero encountered in divide
  dA = -(1/m) * ( ( Y/AL ) - ( 1-Y / 1-AL ) )
/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_7311/1066190070.py:6: RuntimeWarning: invalid value encountered in divide
  dA = -(1/m) * ( ( Y/AL ) - ( 1-Y / 1-AL ) )


In [10]:
# Store final layer info
dZ = dZL
dW = dWL
dB = dBL

for l in reversed(range(1, L)):
    print(l)
    # Get dA from information of the front layer
    dA = np.dot( parameters[f"W{l+1}"].T , dZ )

    # Update the parameters of the layer in front of this one via gradient descent now that we no longer need that information
    parameters[f"W{l+1}"] -= learning_rate*dW
    parameters[f"B{l+1}"] -= learning_rate*dB

    # Get dZl
    dZ = dA * activation_prime(cache[f"Z{l}"] , type='relu')

    # Get dWl and dBl
    dW = np.dot( dZ, cache[f"A{l-1}"].T )
    dB = np.sum( dZ, axis=1, keepdims=True)

# Update the first layer 
parameters[f"W{1}"] -= learning_rate*dW
parameters[f"B{1}"] -= learning_rate*dB

3
2
1


/var/folders/kr/b0myfb8n375_bzh05rj27xjm0000gn/T/ipykernel_7311/1685876607.py:16: RuntimeWarning: invalid value encountered in multiply
  dZ = dA * activation_prime(cache[f"Z{l}"] , type='relu')


# Overall Functions

Lets compress this all into an `initialise_network`, `forwards_pass` and `backwards_pass` functions so that it is much easier to read. Along with a `train_neural_network` function that will wrap all of them.

Can't really implement an `update_final_layer` and `update_hidden_layer` function since we need the information on the parameters of the forward layer to calculate derivatives before we can update it using gradient descent.

In [13]:
def intialise_network(
        layer_dims: list
):
    """Initialise the neural network

    Args:
        X (np.array): Matrix containing the training data. Dimension (features, examples)
        layer_dims (list): How many neurons each layer should have. First element must be the number of features in the training data.

    Returns:
        parameters (dict): The initialised weights and bias matrices.
    """
    L = len(layer_dims) - 1
    m = X.shape[1]

    parameters = {}
    for l in range (1, L+1):
        parameters[f"W{l}"] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters[f"B{l}"] = np.zeros((layer_dims[l], m))

    return parameters

In [12]:
def forwards_pass(
        X: np.array,
        parameters: dict,
        layer_dims: list
):
    """Perform a forward pass through the neural network.

    Args:
        X (np.array): Data matrix of dimension (features, examples).
        parameters (dict): Weight and bias matrices throughout the network.
        layer_dims (list): The number of neurons in each layer of the network.

    Returns:
        cache (dict): The linear and activation matrices for each layer in the network.
    """
    L = len(layer_dims) - 1
    cache = {}
    cache['A0'] = X

    # All hidden layers use ReLU
    for l in range(1, L):
        cache[f"Z{l}"] = np.dot(parameters[f"W{l}"], cache[f"A{l-1}"]) + parameters[f"B{l}"]
        cache[f"A{l}"] = relu(cache[f"Z{l}"])

    # Do final layer with sigmoid 
    cache[f"Z{L}"] = np.dot(parameters[f"W{L}"], cache[f"A{L-1}"]) + parameters[f"B{L}"]
    cache[f"A{L}"] = sigmoid(cache[f"Z{L}"])

    return cache

In [11]:
def backwards_pass(
        Y: np.array,
        parameters: dict,
        cache: dict,
        L: int,
        learning_rate: float,
        type: str
):
        """Perform backpropagation and implement gradient descent on the parameters in all layers within the network.

        Args:
                Y (np.array): Matrix of the training labels.
                parameters (dict): Contains the weights and biases matrices.
                cache (dict): Contains the linear and activation matrices.
                L (int): How many layers does the network have.
                learning_rate (float): How quickly gradient descent should be performed.
                type (str): The type of problem being solved, Can be 'binary classification', 'multilabel classification' or 'regression'.
        
        Returns:
                parameters (dict): Tuned weights and biases matrices after performing gradient descent.
        """
        # Make a deep copy of parameters 
        parameters = np.copy(parameters, order='k')

        # Do pass on the final layer first
        if type == 'binary classification':
                dZ = dA_final(Y=Y, AL=cache[f'Z{L}'], type=type) * activation_prime(cache[f"Z{L}"], 'sigmoid')
        elif type == 'multilabel classification':
                dZ = dA_final(Y=Y, AL=cache[f'Z{L}'], type=type) * activation_prime(cache[f"Z{L}"], 'softmax')
        elif type == 'regression':
                dZ = dA_final(Y=Y, AL=cache[f'Z{L}'], type=type)
        else:
                raise ValueError("Choose a valid problem type.")
        
        dW = np.dot(dZ, cache[f"A{L-1}"].T)
        dB = np.sum(dZ, axis=1, keepdims=True)

        # Propagate through the hidden layers
        for l in reversed(range(1, L)):
                # Get dA from information of the front layer
                dA = np.dot( parameters[f"W{l+1}"].T , dZ )

                # Update the parameters of the layer in front of this one via gradient descent now that we no longer need that information
                parameters[f"W{l+1}"] -= learning_rate*dW
                parameters[f"B{l+1}"] -= learning_rate*dB

                dZ = dA * activation_prime(cache[f"Z{l}"] , type='relu')

                dW = np.dot( dZ, cache[f"A{l-1}"].T )
                dB = np.sum( dZ, axis=1, keepdims=True)

        # Update the first layer 
        parameters[f"W{1}"] -= learning_rate*dW
        parameters[f"B{1}"] -= learning_rate*dB

        return parameters
